# Demonstration of Creating and Running a NATS input file from IFF data
## Read in IFF Data using PARA-ATM

In [1]:
from paraatm.io import iff, gnats, utils
from paraatm.io.iff import read_iff_file

iff_fname = 'IFF_SFO+ASDEX_20190511_080104_86221.csv'
#iff_fname = 'IFF_SFO_ASDEX_ABC123.csv'

iff_data =  read_iff_file(iff_fname,record_types=[2,3,4,8])
print(iff_data[3].columns)
print(len(iff_data[3]))

Index(['recType', 'time', 'fltKey', 'bcnCode', 'cid', 'Source', 'msgType',
       'callsign', 'recTypeCat', 'latitude', 'longitude', 'altitude',
       'significance', 'coord1Accur', 'coord2Accur', 'altAccur', 'tas',
       'heading', 'rocd', 'altQualifier', 'altIndicator', 'trackPtStatus',
       'leaderDir', 'scratchPad', 'msawInhibitInd', 'assignedAltString',
       'controllingFac', 'controllingSeg', 'receivingFac', 'receivingSec',
       'activeContr', 'primaryContr', 'kybrdSubset', 'kybrdSymbol', 'adsCode',
       'opsType', 'airportCode', 'trackNumber', 'tptReturnType', 'modeSCode'],
      dtype='object')
1030410


## Clean callsign list of UNKN, nan, 1200, and ground operations vehicles

In [5]:
callsigns = iff_data[3].callsign.unique()
cs = callsigns.tolist()
cs.remove('UNKN') # remove 'UNKN'
cs = [x for x in cs if (str(x) != 'nan')] #remove nan
cs = [x for x in cs if not x.startswith('OPS')] #remove ground operation vehicles
cs.remove('1200')

## Choose path for trx to be generated and NATS results

In [8]:
import os

#Filenames to write modified trx and mfl files to. Need full path because GNATS wrapper changes the directory to the GNATS directory.
trx_dir = '/home/edecarlo/para-atm-collection/miscellaneous/gnats-fpgen/trx'
results_dir = '/home/edecarlo/para-atm-collection/miscellaneous/gnats-fpgen/results'

trx_fname = '/iff_to_gnats_geo'
mfl_file= trx_dir+trx_fname+'_mfl.trx'
trx_file = trx_dir+trx_fname+'.trx'
results_file = results_dir+trx_fname+'.csv'

if os.path.exists(trx_file):
    os.remove(trx_file)
    print('Previously generated trx file removed...')

if os.path.exists(mfl_file):
    os.remove(mfl_file)
    print('Previously generated mfl file removed...')

## Start NATS Simulation in the background

In [16]:
from gnats_gate_to_gate import GateToGate
natsSim = GateToGate()

## Create FlightPlanSelector object

In [19]:
from FlightPlanSelector import FlightPlanSelector
dirPath = natsSim.DIR_share
fpath = dirPath + '/tg/trx/TRX_07132005_noduplicates_crypted'
f=FlightPlanSelector(natsSim,fname=fpath)

In [23]:
f.flmap.keys()

dict_keys(['PHX-EWR', 'LAS-MSP', 'ATL-SEA', 'SFO-JFK', 'SFO-ATL', 'PHNL-KORD', 'SMF-ORD', 'LAX-IAD', 'ABQ-SDF', 'SEA-LAS', 'UCA-PHL*', 'LAS-DTW', 'PHKO-KORD', 'PHX-CLT', 'LAX-JFK', 'LAX-MIA', 'SFO-PHL', 'ATL-PDX', 'SFO-BOS', 'KLAX-EGLL', 'ONT-RFD', 'SMF-IAD', 'SJC-LIT', 'SAN-JFK', 'SDF-OAK', 'PHOG-KORD', 'LAS-ATL', 'BFI-SDF', 'EDDF-KORD', 'KJFK-PANC', 'SJC-SDF', 'IND*-MDW', 'DTW-DVT', 'LAS-IAH', 'SLC-CVG', 'MMMX-EDDF', 'PHL-ABY', 'DAL-MEM', 'LAX-TPA', 'MAF-94E', 'CYVR-CYYZ', 'MKC-MML', 'APA-LCK', 'LAS-BOS', 'EWR-LAX', 'SAN-PHL', 'KLAS-CYYZ', 'SJC-BOS', 'SFO-MSP', 'PWK-OSU', 'MMU-CRW', 'PDX-OAK', 'PHL-PBI', 'BAL-MSY', 'SFO-IAD', 'MSP-FWA', 'LGB-BOS', 'SFO-MDW', 'PHX-DTW', 'SBGR-KORD', 'LAX-PHL', 'LAS-RDU', 'DEN-SDF', 'SDF-DEN', 'SFO-CLE', 'CYWG-CYHM', 'BOS-LCK', 'LAX-CVG', 'SLC-JFK', 'ORD-SEA', 'ELP-FWA', 'MQY-AWM', 'CAE-ABY', 'PNE-LEW', 'CYYC-EGPF', 'SAN-IAD', 'LAS-PHL', 'LGB-IAD', 'APA-DSM', 'OAK-SDF', 'SAN-ORD', 'JFK-CHS', 'LAS-MCO', 'SFO-ORD', 'HNL-ANC', 'RJAA-KJFK', 'ORD-OSC*', 'LA

## Generate NATS TRX flightplans

In [20]:
iffBaseAirport= "KSFO"

from iff_functions import (get_departure_airport_from_iff,
                           get_arrival_airport_from_iff,
                           get_departure_gate_and_rwy_from_iff,
                           get_arrival_gate_and_rwy_from_iff,
                           random_airport_gate_and_rwy,
                           check_if_flight_has_departed,
                           check_if_flight_landing_in_dataset)

In [21]:
for i,callsign in enumerate(cs[:2]):
    flightInAir = check_if_flight_has_departed(iff_data,callsign,natsSim,iffBaseAirport)
    flightLandingInData = check_if_flight_landing_in_dataset(iff_data,callsign,natsSim,iffBaseAirport)
    
    arrivalAirport = ''
    departureAirport=''
    arrivalRwy = ''
    arrivalGate = ''
    departureRwy =''
    departureGate = ''

    if flightInAir and flightLandingInData:
        #Get departure airport. If none is known in the IFF+ASDEX file (i.e., it is not the airport whose name is in the iff_fname), then set departure airport as closest airport to the first lat/lon in the dataset. In the future, would like to use IFF_USA to determine departureAirport in this case
        print (callsign, i)
        arrivalAirport = 'KSFO'
        departureAirport = get_departure_airport_from_iff(iff_data,callsign,natsSim,arrivalAirport=iffBaseAirport,flmap=f.flmap)
        arrivalRwy,arrivalGate= get_arrival_gate_and_rwy_from_iff(iff_data,callsign,natsSim,arrivalAirport)
        result_generated4 = f.generate(4, departureAirport, arrivalAirport, "", arrivalGate, "", arrivalRwy)
        print(result_generated4[0])

    if  not flightInAir and not flightLandingInData:
        print (callsign, i)
        departureAirport = 'KSFO'
        arrivalAirport = get_arrival_airport_from_iff(iff_data,callsign,natsSim,departureAirport,f.flmap)
        departureRwy,departureGate = get_departure_gate_and_rwy_from_iff(iff_data,callsign,natsSim,departureAirport) #doesn't exist
        arrivalRwy,arrivalGate = random_airport_gate_and_rwy(natsSim,arrivalAirport)
        result_generated1 = f.generate(1, departureAirport, arrivalAirport, departureGate, arrivalGate, departureRwy, arrivalRwy)
        print(result_generated1[0]) 

DAL2312 0
ATL.<>.CARPT..BNA.J46.ARG..RZC..PER..GCK.J28.MLF.J80.OAL.RW10L.<{"id": "Rwy_04_001"}, {"id": "Txy_C_002"}, {"id": "Txy_C_001"}, {"id": "Rwy_03_001"}, {"id": "Txy_F_001"}, {"id": "Txy_F_002"}, {"id": "Txy_F_003"}, {"id": "Txy_F_004"}, {"id": "Txy_F_N"}, {"id": "Txy_F_005"}, {"id": "Txy_F_L"}, {"id": "Txy_F1_004"}, {"id": "Rwy_02_005"}, {"id": "Txy_F1_003"}, {"id": "Txy_F1_002"}, {"id": "Rwy_01_004"}, {"id": "Txy_F1_001"}, {"id": "Txy_B_F1"}, {"id": "Txy_A_F1"}, {"id": "Txy_A_008"}, {"id": "Txy_A_G"}, {"id": "Txy_A_007"}, {"id": "Ramp_03_004"}, {"id": "Ramp_03_003"}, {"id": "Ramp_03_002"}, {"id": "Ramp_03_001"}, {"id": "Gate_B_021"}>.KSFO
UAL2377 1
No viable destination airport found for UAL2377. Returning random from FlightPlanSelector options.
Can't find the landing runway end point
KSFO.<{"id": "Gate_B_032A"}, {"id": "Ramp_02_008"}, {"id": "Txy_A_006"}, {"id": "Txy_A_007"}, {"id": "Txy_A_G"}, {"id": "Txy_A_008"}, {"id": "Txy_A_F1"}, {"id": "Txy_B_F1"}, {"id": "Txy_F1_001"}, 